In [2]:
from bs4 import BeautifulSoup
import urllib3
from urllib.request import urlopen
import smtplib
from flask import Flask, request, Response
import re
import time 
from textblob import TextBlob

In [3]:
def make_soup(url):
    html = urlopen(url)
    return BeautifulSoup(html, 'html.parser')

In [3]:
url = "https://www.pc-kombo.com/us/components/cpus"
soup = make_soup(url)


lista = [m for m in soup.find("body").find("ol", {'id': 'hardware'}).find_all('li')]
final = []
ibi = 0
for i in lista:
    
    i = i.find('a')
    if(i != 'None' and i != -1):
        ibi += 1
        nombre = i.find("h5", {'class': 'name'})
        subtitle = i.find("div", {'class': 'subtitle'}).find_all('span')
        
        print("Nombre: " + str(nombre.get_text(strip=True)))
        #print("Cores: "+ str(cores.get_text(strip=True)))
        print("Socket: ", end = "")
        nombre,socket,reloj,ok,cores,threads
        for j in subtitle:
            j = str(j.get_text(strip=True))
            if(len(j) > 2):
                print(j)
        print("----------------------------------------------------------------")
    
print(ibi)   

Nombre: AMD Ryzen 5 5600X
Socket: AM4
Clock 3.7 GHz
Turbo 4.6 GHz
6Cores
12 Threads
----------------------------------------------------------------
Nombre: AMD Athlon 3000G
Socket: AM4
Clock 3.5 GHz
2Cores
4 Threads
----------------------------------------------------------------
Nombre: AMD Ryzen 3 3300X
Socket: AM4
Clock 3.8 GHz
Turbo 4.3 GHz
4Cores
8 Threads
----------------------------------------------------------------
Nombre: AMD Ryzen 5 5500
Socket: AM4
Clock 3.6 GHz
Turbo 4.2 GHz
6Cores
12 Threads
----------------------------------------------------------------
Nombre: AMD Ryzen 5 5600
Socket: AM4
Clock 3.5 GHz
Turbo 4.4 GHz
6Cores
12 Threads
----------------------------------------------------------------
Nombre: AMD Ryzen 5 5600G
Socket: AM4
Clock 3.9 GHz
Turbo 4.4 GHz
6Cores
12 Threads
----------------------------------------------------------------
Nombre: AMD Ryzen 5 7600
Socket: AM5
Clock 3.8 GHz
Turbo 5.1 GHz
6Cores
12 Threads
------------------------------------------

In [18]:
url = "https://www.pc-kombo.com/us/components/gpus"
soup = make_soup(url)


lista = [m for m in soup.find("body").find("ol", {'id': 'hardware'}).find_all('li')]
final = []
ibi = 0
for i in lista:
    
    i = i.find('a')

    if(i != 'None' and i != -1):
        ibi += 1
        nombre = i.find("h5", {'class': 'name'})
        subtitle = i.find("div", {'class': 'subtitle'}).find_all('span')
        print(i)
        print("Link: " + str(i.find('href')))
        print("Nombre: " + str(nombre.get_text(strip=True)))
        #print("Cores: "+ str(cores.get_text(strip=True)))
        print("Socket: ", end = "")
        for j in subtitle:
            j = str(j.get_text(strip=True))
            if(len(j) > 2):
                print(j)
        print("----------------------------------------------------------------")
    
print(ibi)   

<a href="https://www.pc-kombo.com/us/product/gpu/4895106290662_Sapphire Pulse Radeon RX 6600 Gaming" target="_blank">
<h5 class="name">Sapphire Pulse Radeon RX 6600 Gaming</h5>
<div class="subtitle">
<span class="series">Radeon RX 6600</span>
<span class="vram">8 GB</span>
<span>132W</span>
</div>
</a>
Link: None
Nombre: Sapphire Pulse Radeon RX 6600 Gaming
Socket: Radeon RX 6600
8 GB
132W
----------------------------------------------------------------
<a href="https://www.pc-kombo.com/us/product/gpu/4719331307844_Aorus RTX 3070 Master" target="_blank">
<h5 class="name">Aorus RTX 3070 Master</h5>
<div class="subtitle">
<span class="series">GeForce RTX 3070</span>
<span class="vram">8 GB</span>
<span>220W</span>
</div>
</a>
Link: None
Nombre: Aorus RTX 3070 Master
Socket: GeForce RTX 3070
8 GB
220W
----------------------------------------------------------------
<a href="https://www.pc-kombo.com/us/product/gpu/4711081635321_ASUS GeForce RTX 3050 Dual" target="_blank">
<h5 class="name">

Socket: GeForce RTX 3070
8 GB
240W
----------------------------------------------------------------
<a href="https://www.pc-kombo.com/us/product/gpu/4719072850777_MSI GeForce RTX 3070 Suprim X 8G LHR" target="_blank">
<h5 class="name">MSI GeForce RTX 3070 Suprim X 8G LHR</h5>
<div class="subtitle">
<span class="series">GeForce RTX 3070</span>
<span class="vram">8 GB</span>
<span>280W</span>
</div>
</a>
Link: None
Nombre: MSI GeForce RTX 3070 Suprim X 8G LHR
Socket: GeForce RTX 3070
8 GB
280W
----------------------------------------------------------------
<a href="https://www.pc-kombo.com/us/product/gpu/4719072843120_MSI GeForce RTX 3070 Ti Gaming X Trio" target="_blank">
<h5 class="name">MSI GeForce RTX 3070 Ti Gaming X Trio</h5>
<div class="subtitle">
<span class="series">GeForce RTX 3070 Ti</span>
<span class="vram">8 GB</span>
<span>310W</span>
</div>
</a>
Link: None
Nombre: MSI GeForce RTX 3070 Ti Gaming X Trio
Socket: GeForce RTX 3070 Ti
8 GB
310W
--------------------------------

Socket: GeForce RTX 2060 Super
8 GB
175W
----------------------------------------------------------------
<a href="https://www.pc-kombo.com/us/product/gpu/4718017553049_ASUS GeForce RTX 2060 Super Dual O8G EVO V" target="_blank">
<h5 class="name">ASUS GeForce RTX 2060 Super Dual O8G EVO V</h5>
<div class="subtitle">
<span class="series">GeForce RTX 2060 Super</span>
<span class="vram">8 GB</span>
<span>175W</span>
</div>
</a>
Link: None
Nombre: ASUS GeForce RTX 2060 Super Dual O8G EVO V
Socket: GeForce RTX 2060 Super
8 GB
175W
----------------------------------------------------------------
<a href="https://www.pc-kombo.com/us/product/gpu/4718017495042_ASUS GeForce RTX 2060 Super ROG Strix 8G EVO" target="_blank">
<h5 class="name">ASUS GeForce RTX 2060 Super ROG Strix 8G EVO</h5>
<div class="subtitle">
<span class="series">GeForce RTX 2060 Super</span>
<span class="vram">8 GB</span>
<span>175W</span>
</div>
</a>
Link: None
Nombre: ASUS GeForce RTX 2060 Super ROG Strix 8G EVO
Socket: Ge

</a>
Link: None
Nombre: EVGA GeForce GTX 1080 Ti SC2 Hybrid Gaming iCX
Socket: GeForce GTX 1080 Ti
11 GB
250W
----------------------------------------------------------------
<a href="https://www.pc-kombo.com/us/product/gpu/4250812437853_EVGA GeForce GTX 1650 KO Ultra Gaming" target="_blank">
<h5 class="name">EVGA GeForce GTX 1650 KO Ultra Gaming</h5>
<div class="subtitle">
<span class="series">GeForce GTX 1650 (4GB)</span>
<span class="vram">4 GB</span>
<span>75W</span>
</div>
</a>
Link: None
Nombre: EVGA GeForce GTX 1650 KO Ultra Gaming
Socket: GeForce GTX 1650 (4GB)
4 GB
75W
----------------------------------------------------------------
<a href="https://www.pc-kombo.com/us/product/gpu/4250812433015_EVGA GeForce GTX 1650 SC Ultra Gaming" target="_blank">
<h5 class="name">EVGA GeForce GTX 1650 SC Ultra Gaming</h5>
<div class="subtitle">
<span class="series">GeForce GTX 1650 (4GB)</span>
<span class="vram">4 GB</span>
<span>85W</span>
</div>
</a>
Link: None
Nombre: EVGA GeForce GTX 1

Socket: GeForce RTX 2080 Ti
11 GB
250W
----------------------------------------------------------------
<a href="https://www.pc-kombo.com/us/product/gpu/4719331303570_Gigabyte GeForce RTX 2080 Ti WindForce OC 11G" target="_blank">
<h5 class="name">Gigabyte GeForce RTX 2080 Ti WindForce OC 11G</h5>
<div class="subtitle">
<span class="series">GeForce RTX 2080 Ti</span>
<span class="vram">11 GB</span>
<span>260W</span>
</div>
</a>
Link: None
Nombre: Gigabyte GeForce RTX 2080 Ti WindForce OC 11G
Socket: GeForce RTX 2080 Ti
11 GB
260W
----------------------------------------------------------------
<a href="https://www.pc-kombo.com/us/product/gpu/4719331303587_Gigabyte GeForce RTX 2080 WindForce OC 8G" target="_blank">
<h5 class="name">Gigabyte GeForce RTX 2080 WindForce OC 8G</h5>
<div class="subtitle">
<span class="series">GeForce RTX 2080</span>
<span class="vram">8 GB</span>
<span>225W</span>
</div>
</a>
Link: None
Nombre: Gigabyte GeForce RTX 2080 WindForce OC 8G
Socket: GeForce RTX 20

Nombre: MSI GeForce RTX 3080 Suprim X 12G LHR
Socket: GeForce RTX 3080 (12GB)
12 GB
390W
----------------------------------------------------------------
<a href="https://www.pc-kombo.com/us/product/gpu/4719072926229_MSI GeForce RTX 3080 Ventus 3X Plus 12G OC LHR" target="_blank">
<h5 class="name">MSI GeForce RTX 3080 Ventus 3X Plus 12G OC LHR</h5>
<div class="subtitle">
<span class="series">GeForce RTX 3080 (12GB)</span>
<span class="vram">12 GB</span>
<span>390W</span>
</div>
</a>
Link: None
Nombre: MSI GeForce RTX 3080 Ventus 3X Plus 12G OC LHR
Socket: GeForce RTX 3080 (12GB)
12 GB
390W
----------------------------------------------------------------
<a href="https://www.pc-kombo.com/us/product/gpu/4711377027298_MSI GeForce RTX 4070 Ti Gaming X Trio" target="_blank">
<h5 class="name">MSI GeForce RTX 4070 Ti Gaming X Trio</h5>
<div class="subtitle">
<span class="series">GeForce RTX 4070 Ti</span>
<span class="vram">12 GB</span>
<span>750W</span>
</div>
</a>
Link: None
Nombre: MSI GeF

</a>
Link: None
Nombre: ZOTAC GAMING GeForce RTX 2060
Socket: GeForce RTX 2060 (6GB)
6 GB
160W
----------------------------------------------------------------
<a href="https://www.pc-kombo.com/us/product/gpu/4895173617805_ZOTAC GAMING GeForce RTX 2060 AMP! Edition" target="_blank">
<h5 class="name">ZOTAC GAMING GeForce RTX 2060 AMP! Edition</h5>
<div class="subtitle">
<span class="series">GeForce RTX 2060 (6GB)</span>
<span class="vram">6 GB</span>
<span>170W</span>
</div>
</a>
Link: None
Nombre: ZOTAC GAMING GeForce RTX 2060 AMP! Edition
Socket: GeForce RTX 2060 (6GB)
6 GB
170W
----------------------------------------------------------------
<a href="https://www.pc-kombo.com/us/product/gpu/4895173619014_ZOTAC GAMING GeForce RTX 2060 Super AMP! Edition" target="_blank">
<h5 class="name">ZOTAC GAMING GeForce RTX 2060 Super AMP! Edition</h5>
<div class="subtitle">
<span class="series">GeForce RTX 2060 Super</span>
<span class="vram">8 GB</span>
<span>175W</span>
</div>
</a>
Link: None
N

In [17]:
def scrapping_motherboards():

    url = "https://www.pc-kombo.com/us/components/motherboards"
    soup = make_soup(url)

    lista = [m for m in soup.find("body").find("ol", {'id': 'hardware'}).find_all('li')]
    final = []
    ibi = 0
    for i in lista:

        i = (i.find('a', href=True)['href']).split(" ")[0]

        if(i != 'None' and i != -1):
            ibi+=1
            print(i)

            url_2 = i
            soup_2 = make_soup(url_2)

            price = soup_2.find("body").find("section", {'class': 'column col-12 col-md-12'}).find('span', {'itemprop':'price'})
            
            if price == None:
                print("PRECIO:", price)
            else:
                print("PRECIO:", price.text)
            list_a = soup_2.find("body").find('div', {'class': 'container grid-xl'}).find('div', {'class': 'columns'}).find_all('dd')
            list_b = soup_2.find("body").find('div', {'class': 'container grid-xl'}).find('div', {'class': 'columns'}).find_all('dt')

            for i in range(len(list_a)):
                print(list_b[i].text, " ", list_a[i].text)
            print("----------------------------------------------------------------")

    print(ibi)
    
scrapping_motherboards()

https://www.pc-kombo.com/us/product/mainboard/4719072733667_MSI
PRECIO: 139.99
Producer   MSI
MPN   7C56-002R
EAN   4719072733667
UPC   824142219362
Year   
Socket   AM4
Chipset   B550
Unlocked   
Form Factor   ATX
Memory Type   DDR4
Memory Capacity   128
Ramslots   4
Supported Ramspeeds   
SATA   6
M.2 (PCI-E 3.0)   1
M.2 (PCI-E 4.0)   1
PCI-E 3.0 x1   2
PCI-E 3.0 x4   
PCI-E 3.0 x8   
PCI-E 3.0 x16   1
PCI-E 4.0 x1   
PCI-E 4.0 x4   
PCI-E 4.0 x8   
PCI-E 4.0 x16   1
USB 3 Slots   2
USB 3 Headers   2
USB 3 Type-C   
VGA   0
DVI   0
Display Port   1
HDMI   1
Wifi   <>
Integrated graphics   <>
Product Page   
----------------------------------------------------------------
https://www.pc-kombo.com/us/product/mainboard/4717677320190_ASRock
PRECIO: None
Producer   ASRock
MPN   970 PRO3 R2.0
EAN   4717677320190
UPC   
Year   
Socket   AM3+
Chipset   970
Unlocked   
Form Factor   ATX
Memory Type   DDR3
Memory Capacity   
Ramslots   4
Supported Ramspeeds   
SATA   
M.2 (PCI-E 3.0)   
M.2 (P

PRECIO: None
Producer   ASRock
MPN   FM2A88M PRO3+
EAN   4717677324860
UPC   
Year   
Socket   FM2+
Chipset   A88X
Unlocked   
Form Factor   Micro-ATX
Memory Type   DDR3
Memory Capacity   64
Ramslots   4
Supported Ramspeeds   
SATA   8
M.2 (PCI-E 3.0)   0
M.2 (PCI-E 4.0)   0
PCI-E 3.0 x1   
PCI-E 3.0 x4   
PCI-E 3.0 x8   
PCI-E 3.0 x16   1
PCI-E 4.0 x1   
PCI-E 4.0 x4   
PCI-E 4.0 x8   
PCI-E 4.0 x16   
USB 3 Slots   2
USB 3 Headers   2
USB 3 Type-C   
VGA   1
DVI   1
Display Port   0
HDMI   1
Wifi   <>
Integrated graphics   <>
Product Page   
----------------------------------------------------------------
https://www.pc-kombo.com/us/product/mainboard/4717677329261_ASRock
PRECIO: 51.99
Producer   ASRock
MPN   90-MXB150-A0UAYZ
EAN   4717677329261
UPC   
Year   
Socket   1151
Chipset   H110
Unlocked   
Form Factor   Micro-ATX
Memory Type   DDR4
Memory Capacity   32
Ramslots   2
Supported Ramspeeds   
SATA   4
M.2 (PCI-E 3.0)   0
M.2 (PCI-E 4.0)   0
PCI-E 3.0 x1   
PCI-E 3.0 x4   
PCI-E 

PRECIO: 121.35
Producer   ASRock
MPN   90-MXB3K0-A0UAYZ
EAN   4717677332407
UPC   
Year   
Socket   1151
Chipset   Z270
Unlocked   
Form Factor   Mini-ITX
Memory Type   DDR4
Memory Capacity   32
Ramslots   2
Supported Ramspeeds   
SATA   6
M.2 (PCI-E 3.0)   1
M.2 (PCI-E 4.0)   0
PCI-E 3.0 x1   
PCI-E 3.0 x4   
PCI-E 3.0 x8   
PCI-E 3.0 x16   1
PCI-E 4.0 x1   
PCI-E 4.0 x4   
PCI-E 4.0 x8   
PCI-E 4.0 x16   
USB 3 Slots   6
USB 3 Headers   2
USB 3 Type-C   
VGA   0
DVI   0
Display Port   0
HDMI   0
Wifi   <>
Integrated graphics   <>
Product Page   
----------------------------------------------------------------
https://www.pc-kombo.com/us/product/mainboard/4717677334180_ASRock
PRECIO: 134.75
Producer   ASRock
MPN   90-MXB5X0-A0UAYZ
EAN   4717677334180
UPC   
Year   
Socket   1151
Chipset   Z370
Unlocked   
Form Factor   ATX
Memory Type   DDR4
Memory Capacity   64
Ramslots   4
Supported Ramspeeds   
SATA   6
M.2 (PCI-E 3.0)   2
M.2 (PCI-E 4.0)   0
PCI-E 3.0 x1   3
PCI-E 3.0 x4   
PCI-E 

PRECIO: None
Producer   ASUS
MPN   90MB0L70-M0EAY0
EAN   4716659985266
UPC   
Year   
Socket   1150
Chipset   H81
Unlocked   
Form Factor   Micro-ATX
Memory Type   DDR3
Memory Capacity   16
Ramslots   2
Supported Ramspeeds   
SATA   2
M.2 (PCI-E 3.0)   0
M.2 (PCI-E 4.0)   0
PCI-E 3.0 x1   
PCI-E 3.0 x4   
PCI-E 3.0 x8   
PCI-E 3.0 x16   
PCI-E 4.0 x1   
PCI-E 4.0 x4   
PCI-E 4.0 x8   
PCI-E 4.0 x16   
USB 3 Slots   2
USB 3 Headers   2
USB 3 Type-C   
VGA   0
DVI   0
Display Port   0
HDMI   0
Wifi   <>
Integrated graphics   <>
Product Page   
----------------------------------------------------------------
https://www.pc-kombo.com/us/product/mainboard/4712900142464_ASUS
PRECIO: None
Producer   Asus
MPN   90MB0NS0-M0EAY0
EAN   4712900142464
UPC   
Year   
Socket   1151
Chipset   Z170
Unlocked   
Form Factor   Mini-ITX
Memory Type   DDR4
Memory Capacity   
Ramslots   2
Supported Ramspeeds   
SATA   
M.2 (PCI-E 3.0)   
M.2 (PCI-E 4.0)   0
PCI-E 3.0 x1   
PCI-E 3.0 x4   
PCI-E 3.0 x8   
PCI

PRECIO: None
Producer   ASUS
MPN   90MB12W0-M0EAY0
EAN   4718017667838
UPC   
Year   
Socket   1200
Chipset   Z490
Unlocked   
Form Factor   Micro-ATX
Memory Type   DDR4
Memory Capacity   128
Ramslots   4
Supported Ramspeeds   
SATA   5
M.2 (PCI-E 3.0)   2
M.2 (PCI-E 4.0)   0
PCI-E 3.0 x1   2
PCI-E 3.0 x4   
PCI-E 3.0 x8   
PCI-E 3.0 x16   2
PCI-E 4.0 x1   
PCI-E 4.0 x4   
PCI-E 4.0 x8   
PCI-E 4.0 x16   
USB 3 Slots   2
USB 3 Headers   4
USB 3 Type-C   
VGA   0
DVI   0
Display Port   0
HDMI   0
Wifi   <>
Integrated graphics   <>
Product Page   
----------------------------------------------------------------
https://www.pc-kombo.com/us/product/mainboard/4711081469698_ASUS
PRECIO: None
Producer   ASUS
MPN   90MB19Q0-M0EAY0
EAN   4711081469698
UPC   
Year   
Socket   1700
Chipset   Z690
Unlocked   
Form Factor   ATX
Memory Type   DDR5
Memory Capacity   128
Ramslots   4
Supported Ramspeeds   
SATA   4
M.2 (PCI-E 3.0)   
M.2 (PCI-E 4.0)   1
PCI-E 3.0 x1   1
PCI-E 3.0 x4   
PCI-E 3.0 x8   

...

PRECIO: 74.99
Producer   ASRock
MPN   90-MXB730-A0UAYZ
EAN   4717677335156
UPC   
Year   
Socket   1151
Chipset   B360
Unlocked   
Form Factor   Micro-ATX
Memory Type   DDR4
Memory Capacity   32
Ramslots   2
Supported Ramspeeds   
SATA   6
M.2 (PCI-E 3.0)   1
M.2 (PCI-E 4.0)   0
PCI-E 3.0 x1   2
PCI-E 3.0 x4   
PCI-E 3.0 x8   
PCI-E 3.0 x16   1
PCI-E 4.0 x1   
PCI-E 4.0 x4   
PCI-E 4.0 x8   
PCI-E 4.0 x16   
USB 3 Slots   
USB 3 Headers   2
USB 3 Type-C   
VGA   0
DVI   0
Display Port   0
HDMI   0
Wifi   <>
Integrated graphics   <>
Product Page   
----------------------------------------------------------------
https://www.pc-kombo.com/us/product/mainboard/4717677336238_ASRock
PRECIO: 129.99
Producer   ASRock
MPN   90-MXB870-A0UAYZ
EAN   4717677336238
UPC   
Year   
Socket   AM4
Chipset   B450
Unlocked   
Form Factor   Mini-ITX
Memory Type   DDR4
Memory Capacity   32
Ramslots   2
Supported Ramspeeds   
SATA   4
M.2 (PCI-E 3.0)   1
M.2 (PCI-E 4.0)   0
PCI-E 3.0 x1   
PCI-E 3.0 x4   
PCI

PRECIO: 116.99
Producer   ASRock
MPN   90-MXBDQ0-A0UAYZ
EAN   4710483931291
UPC   
Year   
Socket   1200
Chipset   B460
Unlocked   
Form Factor   Micro-ATX
Memory Type   DDR4
Memory Capacity   128
Ramslots   4
Supported Ramspeeds   
SATA   6
M.2 (PCI-E 3.0)   1
M.2 (PCI-E 4.0)   
PCI-E 3.0 x1   1
PCI-E 3.0 x4   
PCI-E 3.0 x8   
PCI-E 3.0 x16   2
PCI-E 4.0 x1   
PCI-E 4.0 x4   
PCI-E 4.0 x8   
PCI-E 4.0 x16   
USB 3 Slots   4
USB 3 Headers   
USB 3 Type-C   1
VGA   0
DVI   0
Display Port   1
HDMI   1
Wifi   <>
Integrated graphics   <>
Product Page   
----------------------------------------------------------------
https://www.pc-kombo.com/us/product/mainboard/4710483931307_ASRock
PRECIO: 74.99
Producer   ASRock
MPN   90-MXBD30-A0UAYZ
EAN   4710483931307
UPC   
Year   
Socket   1200
Chipset   B460
Unlocked   
Form Factor   Micro-ATX
Memory Type   DDR4
Memory Capacity   64
Ramslots   2
Supported Ramspeeds   
SATA   4
M.2 (PCI-E 3.0)   1
M.2 (PCI-E 4.0)   0
PCI-E 3.0 x1   2
PCI-E 3.0 x4   

PRECIO: 72.99
Producer   ASRock
MPN   90-MXBDJ0-A0UAYZ
EAN   4710483931635
UPC   
Year   
Socket   AM4
Chipset   B550
Unlocked   
Form Factor   Micro-ATX
Memory Type   DDR4
Memory Capacity   64
Ramslots   2
Supported Ramspeeds   
SATA   4
M.2 (PCI-E 3.0)   
M.2 (PCI-E 4.0)   1
PCI-E 3.0 x1   1
PCI-E 3.0 x4   
PCI-E 3.0 x8   
PCI-E 3.0 x16   
PCI-E 4.0 x1   
PCI-E 4.0 x4   
PCI-E 4.0 x8   
PCI-E 4.0 x16   1
USB 3 Slots   4
USB 3 Headers   2
USB 3 Type-C   
VGA   1
DVI   1
Display Port   0
HDMI   1
Wifi   <>
Integrated graphics   <>
Product Page   
----------------------------------------------------------------
https://www.pc-kombo.com/us/product/mainboard/4710483931727_ASRock
PRECIO: 129.99
Producer   ASRock
MPN   90-MXBDH0-A0UAYZ
EAN   4710483931727
UPC   
Year   
Socket   AM4
Chipset   B550
Unlocked   
Form Factor   Mini-ITX
Memory Type   DDR4
Memory Capacity   64
Ramslots   2
Supported Ramspeeds   
SATA   4
M.2 (PCI-E 3.0)   
M.2 (PCI-E 4.0)   1
PCI-E 3.0 x1   
PCI-E 3.0 x4   
PCI-E

PRECIO: 96.99
Producer   ASRock
MPN   90-MXBHV0-A0UAYZ
EAN   4710483936616
UPC   
Year   
Socket   1700
Chipset   B660
Unlocked   
Form Factor   ATX
Memory Type   DDR4
Memory Capacity   128
Ramslots   4
Supported Ramspeeds   
SATA   6
M.2 (PCI-E 3.0)   
M.2 (PCI-E 4.0)   1
PCI-E 3.0 x1   2
PCI-E 3.0 x4   
PCI-E 3.0 x8   
PCI-E 3.0 x16   1
PCI-E 4.0 x1   
PCI-E 4.0 x4   
PCI-E 4.0 x8   
PCI-E 4.0 x16   1
USB 3 Slots   4
USB 3 Headers   
USB 3 Type-C   
VGA   0
DVI   0
Display Port   1
HDMI   1
Wifi   
Integrated graphics   <>
Product Page   
----------------------------------------------------------------
https://www.pc-kombo.com/us/product/mainboard/4710483937057_ASRock
PRECIO: 119.99
Producer   ASRock
MPN   90-MXBH20-A0UAYZ
EAN   4710483937057
UPC   
Year   
Socket   1700
Chipset   B660
Unlocked   
Form Factor   ATX
Memory Type   DDR4
Memory Capacity   128
Ramslots   4
Supported Ramspeeds   
SATA   6
M.2 (PCI-E 3.0)   
M.2 (PCI-E 4.0)   2
PCI-E 3.0 x1   2
PCI-E 3.0 x4   
PCI-E 3.0 x8 

KeyboardInterrupt: 

In [ ]:
url = "https://www.pc-kombo.com/us/product/motherboard/4719072733667_MSI%20B550-A%20Pro"
soup = make_soup(url)

price = soup.find("body").find("section", {'class': 'column col-12 col-md-12'}).find('span', {'itemprop':'price'}).text
print("PRECIO:", price)
list_a = soup.find("body").find('div', {'class': 'container grid-xl'}).find('div', {'class': 'columns'}).find_all('dd')
list_b = soup.find("body").find('div', {'class': 'container grid-xl'}).find('div', {'class': 'columns'}).find_all('dt')

for i in range(len(list_a)):
    print(list_b[i].text, " ", list_a[i].text)
    

In [18]:
def scrapping_all_components(code):

    url = "https://www.pc-kombo.com/us/components/" + code
    soup = make_soup(url)

    lista = [m for m in soup.find("body").find("ol", {'id': 'hardware'}).find_all('li')]
    final = []
    ibi = 0
    for i in lista:

        i = (i.find('a', href=True)['href']).split(" ")[0]

        if(i != 'None' and i != -1):
            ibi+=1
            print(i)

            url_2 = i
            soup_2 = make_soup(url_2)

            price = soup_2.find("body").find("section", {'class': 'column col-12 col-md-12'}).find('span', {'itemprop':'price'})
            
            if price == None:
                print("PRECIO:", price)
            else:
                print("PRECIO:", price.text)
            list_a = soup_2.find("body").find('div', {'class': 'container grid-xl'}).find('div', {'class': 'columns'}).find_all('dd')
            list_b = soup_2.find("body").find('div', {'class': 'container grid-xl'}).find('div', {'class': 'columns'}).find_all('dt')

            for i in range(len(list_a)):
                print(list_b[i].text, " ", list_a[i].text)
            print("----------------------------------------------------------------")

    print(ibi)
    
scrapping_motherboards("gpus")

https://www.pc-kombo.com/us/product/gpu/4895106290662_Sapphire
PRECIO: 279.99
Producer   SAPPHIRE
MPN   11310-01-20G
EAN   4895106290662
UPC   
Year   
Length   193 mm
Slots   2.5
8-pin connectors   1
6-pin connectors   0
HDMI   1
DisplayPort   3
DVI   0
VGA   0
Boost Clock   2044 MHz
Vram   8 GB
Memory Clock   0 MHz
Sync   
TDP   132 W
Product Page   
----------------------------------------------------------------
https://www.pc-kombo.com/us/product/gpu/4719331307844_Aorus
PRECIO: 629.99
Producer   Gigabyte
MPN   GV-N3070AORUS M-8GD
EAN   4719331307844
UPC   
Year   
Length   290 mm
Slots   3.0
8-pin connectors   2
6-pin connectors   0
HDMI   3
DisplayPort   3
DVI   0
VGA   0
Boost Clock   1845 MHz
Vram   8 GB
Memory Clock   14000 MHz
Sync   G-Sync
TDP   220 W
Product Page   
----------------------------------------------------------------
https://www.pc-kombo.com/us/product/gpu/4711081635321_ASUS
PRECIO: None
Producer   ASUS
MPN   90YV0HH0-M0NA00
EAN   4711081635321
UPC   
Year   
L

PRECIO: None
Producer   ASUS
MPN   90YV0FQQ-M0NA00
EAN   4711081452416
UPC   
Year   
Length   310 mm
Slots   4.3
8-pin connectors   2
6-pin connectors   0
HDMI   2
DisplayPort   3
DVI   0
VGA   0
Boost Clock   1815 MHz
Vram   8 GB
Memory Clock   14000 MHz
Sync   
TDP   220 W
Product Page   
----------------------------------------------------------------
https://www.pc-kombo.com/us/product/gpu/4711081320302_ASUS
PRECIO: None
Producer   ASUS
MPN   90YV0FRA-M0NA00
EAN   4711081320302
UPC   
Year   
Length   318.5 mm
Slots   2.9
8-pin connectors   2
6-pin connectors   0
HDMI   2
DisplayPort   3
DVI   0
VGA   0
Boost Clock    MHz
Vram   8 GB
Memory Clock   14000 MHz
Sync   
TDP   220 W
Product Page   
----------------------------------------------------------------
https://www.pc-kombo.com/us/product/gpu/4711081320289_ASUS
PRECIO: None
Producer   ASUS
MPN   90YV0FR9-M0NA00
EAN   4711081320289
UPC   
Year   
Length   318.5 mm
Slots   2.9
8-pin connectors   2
6-pin connectors   0
HDMI   2
D

KeyboardInterrupt: 